In [2]:
import os 
from pathlib import Path
import collections
import shutil
import ssl 
import copy
import math
import timm
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from datetime import datetime 
import albumentations as A
import cv2 as cv 
from sklearn.model_selection import StratifiedKFold
from PIL import Image
from albumentations.pytorch.transforms import ToTensorV2
import torch
from torch import tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset,DataLoader,random_split,Subset
from torchvision import datasets,models,transforms
ssl._create_default_https_context = ssl._create_unverified_context 
import PIL          

**Background**:Rice (Oryza sativa) is one of the staple foods worldwide. Paddy, the raw grain before removal of husk, is cultivated in tropical climates, mainly in Asian countries. Paddy cultivation requires consistent supervision because several diseases and pests might affect the paddy crops, leading to up to 70% yield loss. Expert supervision is usually necessary to mitigate these diseases and prevent crop loss. With the limited availability of crop protection experts, manual disease diagnosis is tedious and expensive. Thus, it is increasingly important to automate the disease identification process by leveraging computer vision-based techniques that achieved promising results in various domains.

**Objective**:Develop a machine or deep learning-based model to classify the given paddy leaf images accurately.

**Type**:Classification.

**Scale**:
training dataset of 10,407 (75%) labeled images across ten classes (nine disease categories and normal leaf) and additional metadata for each image, such as the paddy variety and age.

test dataset of 3,469 (25%) images into one of the nine disease categories or a normal leaf.

**Evaluation**:Accuracy.

In [5]:
# transforms transform 
path = Path('/Users/wangshuo/Library/Mobile Documents/com~apple~CloudDocs/data/paddy-disease-classification/') 

train_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

class PaddyDataset(Dataset):
    def __init__(self,dataset,transform):
        self.dataset = dataset
        self.transform = transform
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self,idx):
        img, label = self.dataset[idx]
        if self.transform:
            img = self.transform(img)
        return img, label 

In [167]:
# albumentations transform
path = Path('/Users/wangshuo/Library/Mobile Documents/com~apple~CloudDocs/data/paddy-disease-classification/') 
train_transform = A.Compose([
    A.HorizontalFlip(),
    A.VerticalFlip(),
    A.RandomRotate90(),
    A.ImageCompression(),
    A.ShiftScaleRotate(shift_limit=0.2,scale_limit=0.2,rotate_limit=45,border_mode=0),
    A.Resize(height=224,width=224),
    A.CoarseDropout(max_height=int(224*0.4),max_width=int(224*0.4),max_holes=1,p=0.75),
    A.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(height=224,width=224),
    A.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    ToTensorV2() 
])

Test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])

class PaddyDataset(Dataset):
    def __init__(self,dataset,transform):
        self.dataset = dataset
        self.transform = transform
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self,idx):
        img, label = self.dataset[idx]
        img = self.transform(image=np.array(img))['image']
        return img, label 

In [168]:
# data 
data = datasets.ImageFolder(path/'train_images')

skf = StratifiedKFold(5, shuffle=True, random_state=7).split(data,data.targets)

train_loader , test_loader = [], []

for train__idx, test_idx in skf:
    train_data = Subset(data,train__idx)
    test_data = Subset(data,test_idx)
    
    train_data = PaddyDataset(train_data,transform=train_transform)
    test_data = PaddyDataset(test_data,transform=test_transform)
    
    train_loader.append(DataLoader(train_data,batch_size=16,shuffle=True))
    test_loader.append(DataLoader(test_data,batch_size=16,shuffle=True)) 

In [169]:
# train
def get_acc(output, label):
    total = output.shape[0]
    _, pred_label = output.max(1)
    num_correct = (pred_label == label).sum().item()
    return num_correct / total

def train(model, train_data, test_data, num_epochs, optimizer, criterion,scheduler):
    
    best_wts = copy.deepcopy(model.state_dict())
    best_acc = 0
    prev_time = datetime.now()
    
    for epoch in range(num_epochs):
        
        train_loss = 0
        train_acc = 0
        
        model = model.train()
        
        for im, label in train_data:
            # forward
            im = im.to('mps')
            label = label.to('mps')
            
            output = model(im)
            
            loss = criterion(output, label)
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_acc += get_acc(output, label)
            
        scheduler.step()
        cur_time = datetime.now()
        
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s) 
        
        if test_data is not None:
            valid_loss = 0
            valid_acc = 0
            model = model.eval()
            
            for im, label in test_data:
                im = im.to('mps')
                label = label.to('mps')
                output = model(im)
                loss = criterion(output, label)
                valid_loss += loss.item()
                valid_acc += get_acc(output, label)
                epoch_str = (
                "Epoch %d. Train Loss: %f, Train Acc: %f, Valid Loss: %f, Valid Acc: %f, "
                % (epoch, train_loss / len(train_data),
                   train_acc / len(train_data), valid_loss / len(test_data),
                   valid_acc / len(test_data)))
            valid_acc = valid_acc /  len(test_data)
            if valid_acc > best_acc:
                best_acc = valid_acc
                best_wts = copy.deepcopy(model.state_dict())
        else:
            epoch_str = ("Epoch %d. Train Loss: %f, Train Acc: %f, " %
                         (epoch, train_loss / len(train_data),
                          train_acc / len(train_data)))
        prev_time = cur_time
        print(epoch_str + time_str)
    model.load_state_dict(best_wts)
    return best_acc, model

In [170]:
# train_new
def train_new(model, train_data, test_data, num_epochs, optimizer, criterion,scheduler):
    
    best_wts = copy.deepcopy(model.state_dict())
    best_acc = 0
    
    prev_time = datetime.now()
    
    for epoch in range(num_epochs):
        # tain model
        train_loss = 0
        train_acc = 0
        
        model = model.train()
        
        for im, label in train_data:
            
            im = im.to('mps:0')
            label = label.to('mps:0')
            # zero  gradients
            optimizer.zero_grad()
            # forward
            output = model(im)
            # get loss
            loss = criterion(output, label)
            # backward
            loss.backward()
            # update parameters
            optimizer.step()
            
            train_loss += loss.item()
            # accuracy
            _, pred_label = output.max(1)
            num_correct = (pred_label == label).sum().item()    
            train_acc += (num_correct / output.shape[0])
        # update learning rate
        scheduler.step()
        
        cur_time = datetime.now()
        # time spent
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s) 
        
        # accuracy on test data
        valid_loss = 0
        valid_acc = 0
        
        model = model.eval()

        for im, label in test_data:
            
            im = im.to('mps:0')
            label = label.to('mps:0')
            
            output = model(im)       
            loss = criterion(output, label) 
            
            valid_loss += loss.item()
            
            _, pred_label = output.max(1)
            num_correct = (pred_label == label).sum().item()    
            valid_acc += (num_correct / output.shape[0])

        epoch_str = (
            "Epoch %d. Train Loss: %f, Train Acc: %f, Valid Loss: %f, Valid Acc: %f, "
            % (epoch, train_loss / len(train_data),
               train_acc / len(train_data), valid_loss / len(test_data),
               valid_acc / len(test_data)))
        # get the best parameters    
        valid_acc = valid_acc /  len(test_data)
        
        if valid_acc > best_acc:
            best_acc = valid_acc
            best_wts = copy.deepcopy(model.state_dict())
            
        prev_time = cur_time
        
        print(epoch_str + time_str)
        
    model.load_state_dict(best_wts)
    
    return best_acc, model  

In [6]:
# resnet50
best_accury = []
Models = []
model_names = []
for i in range(len(train_loader)):
    model_names.append('model'+str(i))
    Models.append('model'+str(i))
for i in range(len(train_loader)):
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

    model.fc = nn.Linear(model.fc.in_features,10)

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9)

    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[11,16], gamma=0.1)
    
    model = model.to('mps')
    
    best_acc, Models[i] = train(model,train_loader[i],test_loader[i],20,optimizer,criterion,scheduler)
    best_accury.append(best_acc)
    
    print('-'*10)
    
print(pd.DataFrame().assign(models=model_names,best_accury=best_accury)) 

Epoch 0. Train Loss: 1.269723, Train Acc: 0.564851, Valid Loss: 0.875445, Valid Acc: 0.735687, Time 00:04:20
Epoch 1. Train Loss: 0.728675, Train Acc: 0.758253, Valid Loss: 0.450567, Valid Acc: 0.851622, Time 00:04:30
Epoch 2. Train Loss: 0.563484, Train Acc: 0.809501, Valid Loss: 0.362977, Valid Acc: 0.877863, Time 00:04:31
Epoch 3. Train Loss: 0.444500, Train Acc: 0.852207, Valid Loss: 0.312301, Valid Acc: 0.901240, Time 00:04:33
Epoch 4. Train Loss: 0.372592, Train Acc: 0.872073, Valid Loss: 0.302947, Valid Acc: 0.905057, Time 00:04:51
Epoch 5. Train Loss: 0.299837, Train Acc: 0.899328, Valid Loss: 0.247030, Valid Acc: 0.932252, Time 00:05:09
Epoch 6. Train Loss: 0.261324, Train Acc: 0.914347, Valid Loss: 0.220034, Valid Acc: 0.939885, Time 00:05:08
Epoch 7. Train Loss: 0.229877, Train Acc: 0.924904, Valid Loss: 0.232943, Valid Acc: 0.936069, Time 00:05:09
Epoch 8. Train Loss: 0.208341, Train Acc: 0.934141, Valid Loss: 0.168827, Valid Acc: 0.947519, Time 00:05:08
Epoch 9. Train Loss

Epoch 15. Train Loss: 0.066457, Train Acc: 0.978527, Valid Loss: 0.078544, Valid Acc: 0.974714, Time 00:05:08
Epoch 16. Train Loss: 0.063151, Train Acc: 0.978567, Valid Loss: 0.091413, Valid Acc: 0.970420, Time 00:05:10
Epoch 17. Train Loss: 0.061567, Train Acc: 0.980326, Valid Loss: 0.073479, Valid Acc: 0.976622, Time 00:05:08
Epoch 18. Train Loss: 0.061634, Train Acc: 0.980686, Valid Loss: 0.077448, Valid Acc: 0.976145, Time 00:05:09
Epoch 19. Train Loss: 0.059609, Train Acc: 0.979966, Valid Loss: 0.077585, Valid Acc: 0.974714, Time 00:05:08
----------
Epoch 0. Train Loss: 1.315196, Train Acc: 0.552863, Valid Loss: 0.819364, Valid Acc: 0.732347, Time 00:04:53
Epoch 1. Train Loss: 0.757359, Train Acc: 0.751040, Valid Loss: 0.646199, Valid Acc: 0.801050, Time 00:05:10
Epoch 2. Train Loss: 0.564984, Train Acc: 0.812500, Valid Loss: 0.347704, Valid Acc: 0.880248, Time 00:05:09
Epoch 3. Train Loss: 0.441108, Train Acc: 0.853807, Valid Loss: 0.286077, Valid Acc: 0.906011, Time 00:05:10
Epo

convnext_tiny  model.classifier[2] = nn.Linear(model.classifier[2].in_features,10)
vit_l_16  model.heads.head = nn.Linear(model.heads.head.in_features,10)
mobilenet_v3_small  model.classifier[3] = nn.Linear(model.classifier[3].in_features,10)
VGG16  model.classifier[6] = nn.Linear(model.classifier[6].in_features,10)
mobilenet_v2  model.classifier[1] = nn.Linear(model.classifier[1].in_features,10) 
Inception v3  model.AuxLogits.fc = nn.Linear(768, 10)  model.fc = nn.Linear(2048, 10)
googlenet  model.fc = nn.Linear(model.fc.in_features,10)
efficientnet_b0  model.classifier[1] = nn.Linear(model.classifier[1].in_features,10）
resnet50    model.fc = nn.Linear(model.fc.in_features,10)

In [182]:
model = timm.create_model('resnet50',pretrained=True)
model.fc = nn.Linear(model.fc.in_features,10) 

In [191]:
model = timm.create_model('resnet50',pretrained=True)
#         for param in model.parameters():
#             param.requires_grad = False
model.fc = nn.Linear(model.fc.in_features,10)

In [83]:
# Net 
class Head(nn.Module):
    def __init__(self,in_features,out_features):
        super(Head,self).__init__()
        self.head = nn.Linear(in_features=in_features,out_features=out_features)
        
    def forward(self,x):
        return self.head(x)
    
class Net(nn.Module):
    def __init__(
        self,
        base_model = 'resnet50',
        pretrained = True,
        checkpoint_path = None,
        num_classes = 10
    ):
        super(Net,self).__init__()
        
        self.backbone = timm.create_model(base_model,pretrained=pretrained,checkpoint_path=checkpoint_path)
        in_features = self.backbone.get_classifier().in_features
        self.backbone.reset_classifier(num_classes=0,global_pool='avg')
        self.neck = Head(in_features=in_features, out_features=in_features)
        self.head = Head(in_features=in_features, out_features=num_classes)
        
    def forward(self,x):
        x = self.backbone(x)
        x = self.neck(x)
        x = self.head(x)
        x = F.log_softmax(x, dim=1)
        
        return x

In [186]:
model = timm.create_model('efficientnet_b3',pretrained=True)

model.classifier = nn.Linear(model.classifier.in_features,10)

In [171]:
def str_cro():
    best_accury = []
    Models = []
    model_names = []
    
    for i in range(len(train_loader)):
        
        model_names.append('model'+str(i))
        Models.append('model'+str(i))
        
    for i in range(len(train_loader)):  
        
        model = timm.create_model('efficientnet_b3',pretrained=True)
        for param in model.parameters():
            param.requires_grad = False
        model.classifier = nn.Linear(model.classifier.in_features,10)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9)
        scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[11,16], gamma=0.1)
        
        model = model.to('mps:0')


        best_acc, Models[i] = train_new(model,train_loader[i],test_loader[i],20,optimizer,criterion,scheduler)
        best_accury.append(best_acc)

        print('-'*10)

    print(pd.DataFrame().assign(models=model_names,best_accury=best_accury))   

In [ ]:
str_cro() 

In [193]:
total_params = sum(p.numel() for p in model.parameters())

In [ ]:
for im, label in train_loader[0]:

    im = im.to('mps:0')
    label = label.to('mps:0')
    # zero  gradients
    optimizer.zero_grad()
    # forward 
    output = model(im)  
    
    print(output) 

In [157]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9) 

In [156]:
model = timm.create_model('convnext_tiny',pretrained=True)

model.head.fc =  nn.Linear(model.head.fc.in_features,10)

model = model.to('mps:0')  

In [8]:
# save model
torch.save(Models[2].state_dict(),'resnet50')

In [295]:
# load model
model = models.resnet50()
model.fc = nn.Linear(model.fc.in_features,10)
model.load_state_dict(torch.load('resnet50'))
model = model.to('mps')

In [7]:
# prediction
model.eval()
image_id = []
label = []
test_dir = path/'Test_images'

for img in (test_dir.iterdir()):
    img_id = img.name
    img = Test_transform(Image.open(img)).unsqueeze(0).to('mps')
    cat = train_data.dataset.dataset.classes[model(img).argmax().item()]
    image_id.append(img_id)
    label.append(cat)

result = pd.DataFrame().assign(image_id=image_id,label=label)
result.to_csv('result.csv',index=False)

In [3]:
a = 'The "portfolio" repository contains a collection of my personal skills and projects, showcasing my experience and abilities in computer vision, machine learning, and deep learning. It includes recent and past projects that demonstrate my coding capabilities and engineering practice experience. All the projects here are completed independently by me, with detailed documentation. '

In [4]:
len(a)

381